In [1]:
# 보스턴 주택 가격 데이터셋 다운받기

from tensorflow.keras.datasets.boston_housing import load_data

# 데이터를 다운받습니다.
(x_train, y_train), (x_test, y_test) = load_data(path='boston_housing.npz', test_split=0.2, seed=777)

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step


In [2]:
# 데이터 형태 확인하기

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(404, 13) (404,)
(102, 13) (102,)


In [3]:
# 데이터 전처리 및 검증 데이터셋 만들기

import numpy as np

# 데이터 표준화
mean = np.mean(x_train, axis = 0)
std = np.std(x_train, axis = 0)

x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# 검증 데이터셋을 만듭니다.
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.33, random_state = 777)

In [4]:
# 모델 구성하기

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
# 입력 데이터의 형태를 꼭 명시해야 합니다.
# 13차원의 데이터를 입력받고, 64개의 출력을 가지는 첫 번째 Dense층
model.add(Dense(64, activation = 'relu', input_shape = (13, )))
# 32개의 출력을 가지는 Dense층
model.add(Dense(32, activation = 'relu'))
# 하나의 값을 출력합니다.
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# 학습하고 평가하기

history = model.fit(x_train, y_train, epochs = 300, validation_data = (x_val, y_val))
model.evaluate(x_test, y_test)

Epoch 1/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 572.7331 - mae: 22.2008 - val_loss: 592.3599 - val_mae: 22.0714
Epoch 2/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 509.3953 - mae: 20.7113 - val_loss: 566.9250 - val_mae: 21.4829
Epoch 3/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 504.9014 - mae: 20.6636 - val_loss: 538.3383 - val_mae: 20.8159
Epoch 4/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 465.7054 - mae: 19.7422 - val_loss: 503.4839 - val_mae: 19.9960
Epoch 5/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 465.6316 - mae: 19.7265 - val_loss: 461.5867 - val_mae: 18.9779
Epoch 6/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 412.3000 - mae: 18.2805 - val_loss: 410.5166 - val_mae: 17.6849
Epoch 7/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 356.6547 - mae: 16.8041 - val_loss: 348.9663 - val_mae: 16.0428
Epoch 8/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 270.6495 - mae: 14.6904 - val_loss: 280.4271 - val_mae: 14.0779
Epoch 9/300
9/9 ━━━━━━━━━━━━━━

[8.997598648071289, 2.1501474380493164]

In [8]:
# K-폴드 사용하기

from tensorflow.keras.datasets.boston_housing import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import KFold

import numpy as np

(x_train, y_train), (x_test, y_test) = load_data(path='boston_housing.npz', test_split=0.2, seed=777)

# 데이터 표준화
mean = np.mean(x_train, axis = 0)
std = np.std(x_train, axis = 0)
# 여기까지는 전부 동일합니다.
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# K-폴드를 진행해봅니다.
k = 3

# 주어진 데이터셋을 k만큼 등분합니다.
# 여기서는 3이므로 학습데이터셋(404개)을 3등분하여
# 한 개는 검증셋으로, 나머지 두 개는 학습 데이터셋으로 활용합니다.
kfold = KFold(n_splits=k, shuffle = True, random_state=777)

# K-폴드 과정에서 재사용을 위해 모델을 반환하는 함수를 정의합니다.
def get_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape = (13,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

mae_list = []   # 테스트 셋을 평가한 후 결과 mae를 담을 리스트를 선언합니다.

# k번 진행합니다.
for train_index, val_index in kfold.split(x_train):
    # 해당 인덱스는 무작위로 생성됩니다.
    # 무작위로 생성해주는 것은 과대적합을 피할 수 있는 좋은 방법입니다.
    x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # 모델을 불러옵니다.
    model = get_model()

    model.fit(x_train_fold, y_train_fold, epochs = 300, validation_data = (x_val_fold, y_val_fold))

    _, test_mae = model.evaluate(x_test, y_test)
    mae_list.append(test_mae)
    

Epoch 1/300


C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 539.3962 - mae: 21.4661 - val_loss: 568.3454 - val_mae: 21.4129
Epoch 2/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 513.2517 - mae: 20.5662 - val_loss: 534.0190 - val_mae: 20.5851
Epoch 3/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 474.2642 - mae: 19.8107 - val_loss: 495.7729 - val_mae: 19.6351
Epoch 4/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 430.5720 - mae: 18.8561 - val_loss: 451.8473 - val_mae: 18.4860
Epoch 5/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 381.8501 - mae: 17.3648 - val_loss: 401.4145 - val_mae: 17.0936
Epoch 6/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 331.5967 - mae: 15.9174 - val_loss: 345.7073 - val_mae: 15.5157
Epoch 7/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 315.8957 - mae: 15.2212 - val_loss: 286.2183 - val_mae: 13.8383
Epoch 8/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 243.1723 - mae: 13.2247 - val_loss: 226.9255 - val_mae: 12.0572
Epoch 9/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms

In [11]:
print('전체 결과 :', mae_list)
print('평균낸 결과를 최종결과로 사용합니다.', np.mean(mae_list))

전체 결과 : [2.092430353164673, 2.033785581588745, 2.2401998043060303]
평균낸 결과를 최종결과로 사용합니다. 2.122138579686483
